In [4]:
!pip install beautifulsoup4 webdriver_manager


In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,738 kB]
Get:13 https://developer.download.nvidia.com/compu

In [2]:
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver
!pip install chromedriver-autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import sys
import os
import time
import requests
import pandas as pd
from tqdm import tqdm

import chromedriver_autoinstaller  # setup chrome options
from bs4 import BeautifulSoup

import urllib.request
from urllib.request import urlretrieve

In [6]:
chrome_path = "/usr/lib/chromium-browser/chromedriver"

In [19]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

# 크롬 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

In [21]:

# 페이지 열기
url = "https://unipass.customs.go.kr/clip/index.do"
driver.get(url)

# JavaScript가 실행될 시간을 기다리기
driver.implicitly_wait(10)  # 최대 10초 대기

# 페이지의 HTML 가져오기
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# 세계HS 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.CLASS_NAME,"M_ULS0200000000")
click_box.click()

# HS해설서 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="LEFTMENU_LNK_UI-ULS-0202-001Q"]/span')
click_box.click()

# 조회 창을 찾아 click_box 변수에 저장 (By.XPATH 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="searchVo"]/div[2]/footer/button/span')
driver.execute_script("arguments[0].click();", click_box)


# Wait for the table to load
time.sleep(1)  # Adjust the wait time as needed

# Locate the table
table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')

# Parse the table HTML using BeautifulSoup
soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')

# Extract table rows
rows = soup.find_all('tr')

# Initialize an empty DataFrame
data = pd.DataFrame(columns=["구분", "년도", "단위", "HS분류", "내용", "Left Area", "Right Area"])

# # Iterate through the rows and extract data
# for row in rows:
#     cols = row.find_all('td')
#     cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
#     if cols:  # Check if the row has data
#         # Click on the `dtlInfo` link
#         dtl_link = row.find('a', class_='dtlInfo')
#         if dtl_link:
#             dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)
#             dtl_element.click()
#             time.sleep(2)  # Wait for the leftArea and rightArea to load

#             # Extract text from leftArea
#             left_area = driver.find_element(By.ID, 'divLft_tab4')
#             left_area_text = left_area.text.strip()

#             # Extract text from rightArea
#             right_area = driver.find_element(By.ID, 'divRght_tab4')
#             right_area_text = right_area.text.strip()

#             # Go back to the main page to continue scraping
#             # driver.back()
#             time.sleep(2)  # Wait for the page to load

#             # Add data to the DataFrame
#             data.loc[len(data)] = cols + [left_area_text, right_area_text]

# Loop through all 135 pages
for page in tqdm(range(1, 136)):
    print(f"Processing page {page}...")

    # Locate the table
    table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')

    # Parse the table HTML using BeautifulSoup
    soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')

    # Extract table rows
    rows = soup.find_all('tr')

    # Iterate through the rows and extract data
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
        if cols:  # Check if the row has data
            # <td class="ellipsis" title="류"> 요소 찾기
            element = row.find('td', class_='ellipsis', title=True)
            dtl_link = row.find('a', class_='dtlInfo')

            # 조건 확인 및 처리
            if element and element['title'] == '류':
                dtl_element=driver.find_element(By.XPATH, f"//a[text()='{dtl_link.text.strip()}']")

            else:
                # Click on the `dtlInfo` link
                dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)

            driver.execute_script("arguments[0].click();", dtl_element)  # Use JavaScript click to avoid staleness
            time.sleep(0.8)  # Wait for the leftArea and rightArea to load

            # Extract text from leftArea
            left_area = driver.find_element(By.ID, 'divLft_tab4')
            left_area_text = left_area.text.strip()

            # Extract text from rightArea
            right_area = driver.find_element(By.ID, 'divRght_tab4')
            right_area_text = right_area.text.strip()

            # Add data to the DataFrame
            data.loc[len(data)] = cols + [left_area_text, right_area_text]

    # Navigate to the next page
    try:
        wait=WebDriverWait(driver, 10)
        next_page = wait.until(EC.element_to_be_clickable((By.XPATH, f'//div[@class="paging"]//a[@href="#{page + 1}"]')))
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(2)  # Wait for the page to load
    except Exception as e:
        print(f"Could not navigate to page {page + 1}: {e}")
        break

# Display the DataFrame

# Save the data to a CSV file
data.to_csv('HSdescription_data.csv', index=False, encoding='utf-8-sig')

  0%|          | 0/135 [00:00<?, ?it/s]

Processing page 1...


  1%|          | 1/135 [00:11<26:09, 11.72s/it]

Processing page 2...


  1%|▏         | 2/135 [00:23<25:37, 11.56s/it]

...

Processing page 10...


  7%|▋         | 10/135 [01:59<24:21, 11.69s/it]

Processing page 11...


  7%|▋         | 10/135 [02:07<26:33, 12.74s/it]


KeyboardInterrupt: 